In [1]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

In [13]:
# # add actual data here
# xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
# lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
# logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred.csv",DataFrame);
returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final_cleaned.csv",DataFrame);

# # add actual data here
#new_xgb_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/new_xgb_pred.csv",DataFrame);
#returns = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_betting_odds_final_cleaned.csv",DataFrame);

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.253165,0.140449
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.108578,0.039185
3,Australia,Denmark,6.38,3.88,0.429185,0.135501,0.204918,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.289855,0.221729
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.23753,0.168919
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.233645,0.131579
7,Brazil,Switzerland,0.469484,3.57,6.23,0.680511,0.218818,0.138313
8,Cameroon,Brazil,7.93,4.32,0.359712,0.111982,0.18797,0.73545
9,Cameroon,Serbia,4.3,2.77,0.70922,0.188679,0.265252,0.585062


In [14]:
returns[1:5,6:8]

Row,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,Float64,Float64,Float64
1,0.64539,0.253165,0.140449
2,0.892819,0.108578,0.039185
3,0.135501,0.204918,0.6997
4,0.526066,0.289855,0.221729
5,0.632353,0.23753,0.168919


In [15]:
returns_matrix = Matrix(returns[:,3:5]);
# # matrix of probabilities for model's predictions
# xgb_probs = Matrix(xgb_preds[:,3:5]);
# rf_probs = Matrix(rf_preds[:,3:5]);
# lgbm_probs = Matrix(lgbm_preds[:,3:5]);
# logreg_probs = Matrix(logreg_preds[:,3:5]);
new_xgb_probs = Matrix(new_xgb_preds[:,3:5]);
implied_probs = Matrix(returns[:,6:8]);

In [7]:
# check if each row of new_xgb_preds and returns are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != returns[i,1] || new_xgb_preds[i,2] != returns[i,2]
        println("not the same")
    end
end


In [20]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B*z[i,j] for all i and j
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B*z[i,j])
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= B/12*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [21]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(implied_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 482 rows, 288 columns and 1296 nonzeros
Model fingerprint: 0xe11096c9
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [4e-02, 9e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 480 rows and 241 columns
Presolve time: 0.00s
Presolved: 2 rows, 47 columns, 94 nonzeros
Variable types: 0 continuous, 47 integer (46 binary)
Found heuristic solution: objective -134.4683006

Root relaxation: objective -1.165249e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

In [22]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt)

5-element Vector{CartesianIndex{2}}:
 CartesianIndex(11, 1)
 CartesianIndex(2, 3)
 CartesianIndex(19, 3)
 CartesianIndex(30, 3)
 CartesianIndex(40, 3)

In [23]:
# print objective value
println("Expected profit: ", exp_prof)

Expected profit: -116.52492571250006
